# Web Scraping Mercado Livre


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page=1'
response = requests.get(URL)
page_contents = response.text

In [ ]:
with open('mercadolivre.html','w', encoding = "utf-8") as f:
    f.write(page_contents)

In [ ]:
from bs4 import BeautifulSoup
doc = BeautifulSoup(page_contents,'html.parser')

In [ ]:
def get_itens_title(doc):
    title_tags = doc.find_all('p', class_='promotion-item__title')
    titles = []
    for tags in title_tags:
        titles.append(tags.text)
    return titles

In [ ]:
def get_price(doc):
    price_tags = doc.find_all('div', class_ = 'andes-money-amount-combo__main-container')
    price = []
    for tags in price_tags:
        price.append(tags.text.replace('Â',''))
    return price

In [ ]:
def get_doc(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.text,'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(response))
    return doc

In [ ]:
def scrape_multiple_pages(n):
    URL = 'https://www.mercadolivre.com.br/ofertas?container_id=MLB779362-1&page='
    titles,prices= [],[]

    for page in range(1,n+1):
        doc = get_doc(URL + str(page))
        titles.extend(get_itens_title(doc))
        prices.extend(get_price(doc))




    itens = {'TITLE':titles,
                'PRICE':prices
            }
    return pd.DataFrame(itens)

In [ ]:
df_mercado_livre = scrape_multiple_pages(10)
df_mercado_livre["SCRAPY_DATETIME"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_mercado_livre['PRICE'] = df_mercado_livre['PRICE'].str.extract(r'(\d+[\.,]?\d*)')

In [ ]:
df_mercado_livre

,TITLE,PRICE,SCRAPY_DATETIME
0,Creatina Monohidratada 500g 100% Pura Soldiers...,"95,13",2024-05-13 21:00:25
1,Varal De Chão Grande Varal De Roupas 3 Andares...,"99,90",2024-05-13 21:00:25
2,Smart TV LG UHD AI ThinQ 55UR871C0SA-C LCD web...,2.298,2024-05-13 21:00:25
3,Cafeteira Expresso Philco 20 Bar Inox Red Pcf2...,"389,99",2024-05-13 21:00:25
4,G-Tech Nebulizador E Inalador De Rede Vibratór...,"123,90",2024-05-13 21:00:25
...,...,...,...
523,Aspirador De Pó Bas1290v 2 Em 1 1250w Britânia...,"159,90",2024-05-13 21:00:25
524,Martelete Rotativo Rompedor 960w 12 Pçs Furade...,"416,49",2024-05-13 21:00:25
525,Câmbio Driving Force Shifter Preto Logitech G,"350,40",2024-05-13 21:00:25
526,Ventilador De Teto Inverter Wind C/ Controle B...,"240,48",2024-05-13 21:00:25


# Armazenando os dados no banco de dados

In [ ]:
import sqlite3
from sqlalchemy import create_engine

In [ ]:
mercado_livre_schema = """
CREATE TABLE mercado_livre (
TITLE              TEXT,
PRICE              INTEGER,
SCRAPY_DATETIME    TEXT
)

"""

In [ ]:
conn = create_engine( 'sqlite:///mercado_livre.sqlite', echo=False )

In [ ]:
conn = sqlite3.connect('mercado_livre.sqlite')

In [ ]:
df_mercado_livre.to_sql('mercado_livre', con=conn, if_exists='append', index=False )

528

In [ ]:
query = """
SELECT * FROM mercado_livre
"""
df_mercado_livre = pd.read_sql_query( query, conn)
df_mercado_livre

,TITLE,PRICE,SCRAPY_DATETIME
0,Creatina Monohidratada 500g 100% Pura Soldiers...,"95,13",2024-05-13 20:56:18
1,Varal De Chão Grande Varal De Roupas 3 Andares...,"99,90",2024-05-13 20:56:18
2,Smart TV LG UHD AI ThinQ 55UR871C0SA-C LCD web...,2.298,2024-05-13 20:56:18
3,Cafeteira Expresso Philco 20 Bar Inox Red Pcf2...,"389,99",2024-05-13 20:56:18
4,G-Tech Nebulizador E Inalador De Rede Vibratór...,"123,90",2024-05-13 20:56:18
...,...,...,...
1045,Aspirador De Pó Bas1290v 2 Em 1 1250w Britânia...,"159,90",2024-05-13 21:00:25
1046,Martelete Rotativo Rompedor 960w 12 Pçs Furade...,"416,49",2024-05-13 21:00:25
1047,Câmbio Driving Force Shifter Preto Logitech G,"350,40",2024-05-13 21:00:25
1048,Ventilador De Teto Inverter Wind C/ Controle B...,"240,48",2024-05-13 21:00:25


In [ ]:
df_mercado_livre[df_mercado_livre["TITLE"]=="Kit Whey Protein 2kg + Bcaa + Creatina + Shaker"]

,TITLE,PRICE,SCRAPY_DATETIME
